# Latency and Tick - Multi

## Python Imports

In [1]:
import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt

In [2]:
from scipy.optimize import minimize

In [3]:
from functools import partial
import datetime as dtt

In [4]:
from numba import jit, njit
from numba import prange
from numba import types

## Tick Imports

In [5]:
# %matplotlib inline

In [6]:
from tick.hawkes import SimuHawkes, SimuHawkesMulti
from tick.base import TimeFunction
from tick.hawkes import HawkesKernelTimeFunc, HawkesKernelExp

In [7]:
import hawklat as hl

## Multidimensional Hawkes with latency

In [8]:
lat2 = 2

In [9]:
α11 = 0.5
β11 = 1.4
α12 = 0.7
β12 = 1.8
α21 = 0.9
β21 = 2.2
α22 = 0.3
β22 = 1.0
λ01 = 0.6
λ02 = 0.2

In [10]:
def time_funclat(f, support, lat=0, steps=100,
              inter_mode=TimeFunction.InterConstRight):
    t_values = np.linspace(0, support, steps + 1)
    y_values = f(t_values)
    if lat > 0:
        t_values_lat = np.linspace(0, lat, steps)
        y_values_lat = np.zeros(steps)
        t_values_shifted = t_values + lat
        t_all = np.concatenate((t_values_lat, t_values_shifted))
        y_all = np.concatenate((y_values_lat, y_values))
    else:
        t_all = t_values.view()
        y_all = y_values.view()
    return TimeFunction(values=(t_all, y_all),
                        border_type=TimeFunction.Border0,
                        inter_mode=inter_mode)

In [11]:
def ek11(t):
    return α11 * np.exp(-β11 * t)
def ek12(t):
    return α12 * np.exp(-β12 * t)
def ek21(t):
    return α21 * np.exp(-β21 * t)
def ek22(t):
    return α22 * np.exp(-β22 * t)

In [12]:
sup11 = 10
sup12 = 10
sup21 = 10
sup22 = 10

In [13]:
ekc11 = time_funclat(ek11, sup11, lat2)
ekc12 = time_funclat(ek12, sup12, lat2)
ekc21 = time_funclat(ek21, sup21, lat2)
ekc22 = time_funclat(ek22, sup22, lat2)

In [14]:
ektf11 = HawkesKernelTimeFunc(ekc11)
ektf12 = HawkesKernelTimeFunc(ekc12)
ektf21 = HawkesKernelTimeFunc(ekc21)
ektf22 = HawkesKernelTimeFunc(ekc22)

## Simulation definitions

In [15]:
n_paths = 100

In [16]:
end_time_1 = 100
end_time_2 = 1000
end_time_3 = 10000

In [17]:
hawkestf2x2_1 = SimuHawkes(baseline=[λ01, λ02], kernels=[[ektf11, ektf12], [ektf21, ektf22]],
                          end_time=end_time_1, verbose=False, seed=13)
hawkestf2x2_2 = SimuHawkes(baseline=[λ01, λ02], kernels=[[ektf11, ektf12], [ektf21, ektf22]],
                          end_time=end_time_2, verbose=False, seed=13)
hawkestf2x2_3 = SimuHawkes(baseline=[λ01, λ02], kernels=[[ektf11, ektf12], [ektf21, ektf22]],
                          end_time=end_time_3, verbose=False, seed=13)

In [18]:
%%time

multitf2x2_1 = SimuHawkesMulti(hawkestf2x2_1, n_simulations=n_paths)
multitf2x2_1.simulate()

CPU times: user 27.9 s, sys: 444 ms, total: 28.3 s
Wall time: 34.7 s


In [19]:
%%time

multitf2x2_2 = SimuHawkesMulti(hawkestf2x2_2, n_simulations=n_paths)
multitf2x2_2.simulate()

CPU times: user 23.8 s, sys: 367 ms, total: 24.1 s
Wall time: 34.6 s


In [20]:
%%time

multitf2x2_3 = SimuHawkesMulti(hawkestf2x2_3, n_simulations=n_paths)
multitf2x2_3.simulate()

CPU times: user 28.8 s, sys: 1.61 s, total: 30.5 s
Wall time: 1min 44s


In [21]:
M = 2

In [22]:
def_bounds_M = [(1e-10, None)] * M + [(1e-10, None)] * (M ** 2) + [(2e-10, None)] * (M ** 2)

In [23]:
def_x0_M = np.array([0.25] * M + [0.5] * (M ** 2) + [1.0] * (M ** 2))

In [24]:
# %%time

# findθlatm(ts, def_bounds_M, def_x0_M, lat2, method='L-BFGS-B')

In [25]:
%%time

results = hl.findθlatms(multitf2x2_1.timestamps, bounds=def_bounds_M, x0=def_x0_M, τ=lat2, method='L-BFGS-B')
np.mean(results, axis=0)

CPU times: user 8.95 s, sys: 98.8 ms, total: 9.05 s
Wall time: 9.1 s


array([[0.68230047, 0.55215428, 0.72867875, 1.89378793, 1.92106985],
       [0.27896873, 1.02679189, 0.36510127, 2.44541227, 2.61449249]])

In [26]:
%%time

results = hl.findθlatms(multitf2x2_1.timestamps, bounds=def_bounds_M, x0=def_x0_M, τ=lat2, method='SLSQP')
np.mean(results, axis=0)

CPU times: user 6.99 s, sys: 59 ms, total: 7.05 s
Wall time: 7.08 s


array([[0.67868855, 0.55109299, 0.72903459, 1.87390338, 1.93147407],
       [0.28072521, 1.02628703, 0.36669532, 2.43878125, 2.65395131]])

In [28]:
%%time

results = hl.findθlatms(multitf2x2_2.timestamps, bounds=def_bounds_M, x0=def_x0_M, τ=lat2, method='L-BFGS-B')
np.mean(results, axis=0)

CPU times: user 1min 9s, sys: 401 ms, total: 1min 9s
Wall time: 1min 9s


array([[0.62980085, 0.52039406, 0.74778337, 1.4360673 , 1.78078978],
       [0.21471438, 0.97714022, 0.30940711, 2.17825913, 1.04939629]])

In [29]:
%%time

results = hl.findθlatms(multitf2x2_2.timestamps, bounds=def_bounds_M, x0=def_x0_M, τ=lat2, method='SLSQP')
np.mean(results, axis=0)

CPU times: user 1min 7s, sys: 333 ms, total: 1min 7s
Wall time: 1min 7s


array([[0.62979538, 0.52037684, 0.74778943, 1.43599844, 1.7808363 ],
       [0.21472141, 0.9771504 , 0.30940997, 2.17827873, 1.04941341]])

In [30]:
%%time

results = hl.findθlatms(multitf2x2_3.timestamps, bounds=def_bounds_M, x0=def_x0_M, τ=lat2, method='L-BFGS-B')
np.mean(results, axis=0)

CPU times: user 16min 10s, sys: 4.39 s, total: 16min 14s
Wall time: 16min 15s


array([[0.59944249, 0.52133502, 0.74622322, 1.37294924, 1.7595362 ],
       [0.19736354, 0.96888156, 0.30955288, 2.11711623, 1.00217364]])

In [31]:
%%time

results = hl.findθlatms(multitf2x2_3.timestamps, bounds=def_bounds_M, x0=def_x0_M, τ=lat2, method='SLSQP')
np.mean(results, axis=0)

CPU times: user 13min 22s, sys: 3.8 s, total: 13min 26s
Wall time: 13min 24s


array([[0.59943253, 0.52128104, 0.74620689, 1.37277518, 1.75952202],
       [0.19736548, 0.96890481, 0.30957518, 2.11717487, 1.00222318]])